sometimes code is too fast for browser driver. in such cases, try increasing sleep time.

In [1]:
from bs4 import BeautifulSoup as bs 
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
import pandas as pd

pls remember to change this path.

In [2]:
DRIVER_PATH = 'chromedriver_win32/chromedriver.exe'

In [3]:
def element_completely_viewable(driver, elem):
    elem_left_bound = elem.location.get('x')
    elem_top_bound = elem.location.get('y')
    elem_width = elem.size.get('width')
    elem_height = elem.size.get('height')
    elem_right_bound = elem_left_bound + elem_width
    elem_lower_bound = elem_top_bound + elem_height

    win_upper_bound = driver.execute_script('return window.pageYOffset')
    win_left_bound = driver.execute_script('return window.pageXOffset')
    win_width = driver.execute_script('return document.documentElement.clientWidth')
    win_height = driver.execute_script('return document.documentElement.clientHeight')
    win_right_bound = win_left_bound + win_width
    win_lower_bound = win_upper_bound + win_height

    return all((win_left_bound <= elem_left_bound,
                win_right_bound >= elem_right_bound,
                win_upper_bound <= elem_top_bound,
                win_lower_bound >= elem_lower_bound)
              )

# scrolls down the page from top to bottom until the element is visible, then clicks on it.
# (selenium doesn't let you click on stuff that's not on the screen)
def find_and_click(driver, elem):
    y_height = 0
    while not element_completely_viewable(driver, elem):
        driver.execute_script(f"window.scrollTo(0, {y_height})") 
        y_height += 100
        time.sleep(0.5)
    elem.click()

driver = webdriver.Chrome(DRIVER_PATH) 
 
page_url = 'https://www.parliament.gov.sg/history/list-of-mps-by-parliament' 
driver.get(page_url) 
driver.maximize_window() 
driver.implicitly_wait(20) 
time.sleep(2) 

parliament_combobox = driver.find_element(
    by=By.ID,
    value='Parliament'
)

num_parliaments = int(parliament_combobox.get_attribute('childElementCount'))
print(f'Total of {num_parliaments} parliaments')


C:\Users\thisi\AppData\Local\Temp\ipykernel_25356\135018422.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


Total of 14 parliaments


In [4]:
mps = []

for parliament_index in range(num_parliaments - 2, num_parliaments + 1): # parls in [12,14]
    # need this line of code below to prevent stale element reference. not sure why but just keep it. 
    parliament_combobox = driver.find_element(
        by=By.ID,
        value='Parliament'
    )

    current_parliament_option = driver.find_element(
        by=By.XPATH,
        value=f'//*[@id="Parliament"]/option[{parliament_index}]'
    )
    
    find_and_click(driver, parliament_combobox)
    time.sleep(0.5)
    find_and_click(driver, current_parliament_option)
    time.sleep(2)
    
    print(f'Looking at parliament {parliament_index}')
    
    full_table = driver.find_element(
        by=By.XPATH,
        value='/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul'
    )
    
    num_rows = int(full_table.get_attribute('childElementCount'))
    print(f'Found {num_rows} MPs')
    
    mps_this_parliament = []
    
    for i in range(1, num_rows+1):
        name_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[1]/div[2]'
        )
        
        party_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[2]'
        )
        
        legis_assembly_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[3]'
        )
        
        also_in_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[4]'
        )
        
        name, party, legis_assembly, also_in = list(map(
            lambda div: div.get_attribute('innerHTML').strip(),
            [name_div, party_div, legis_assembly_div, also_in_div]
        ))
        
        other_parl_nums = []
        
        # use 0 to denote legislative assembly
        if legis_assembly:
            other_parl_nums.append(0)
        
        for other_parl in also_in.split(', '):
            other_parl_num = int(other_parl[:-2])
            other_parl_nums.append(other_parl_num)
        
        if 'Resigned' in name:
            resigned_index = name.find('Resigned')
            name = name[:resigned_index]
            while not name[-1].isalpha():
                name = name[:-1]
            print(f'found a resigned guy: {name}')
            
        if 'Assoc Prof' in name:
            assoc_index = name.find('Assoc Prof')
            name = name[:assoc_index]
            while not name[-1].isalpha():
                name = name[:-1]
            print(f'found a assoc prof guy: {name}')
            
        if 'Prof' in name:
            prof_index = name.find('Prof')
            name = name[:prof_index]
            while not name[-1].isalpha():
                name = name[:-1]
            print(f'found a prof guy: {name}')
        
        mps_this_parliament.append((name, party, tuple(other_parl_nums)))
    
    mps += mps_this_parliament

Looking at parliament 12
Found 110 MPs
found a resigned guy: Halimah Yacob
found a resigned guy: David Ong Kim Huat
found a resigned guy: Michael Palmer
found a resigned guy: Yaw Shin Leong
Looking at parliament 13
Found 111 MPs
found a resigned guy: Halimah Yacob
found a resigned guy: David Ong Kim Huat
Looking at parliament 14
Found 102 MPs
found a resigned guy: Raeesah Khan


In [5]:
print(f'{len(mps)} entries')
print(f'{len(set(mps))} unique entries')

323 entries
186 unique entries


In [6]:
dataframe = pd.DataFrame(sorted(set(mps)), columns=['Name','Party','Parliaments'])

## Cleaning

PAP has two different representations (different apostrophe).

Number of rows with the slanted apostrophe (’) is 83.

Treatment: Changed to the normal apostrophe (').

In [7]:
dataframe['Party'].unique()

array(['', 'People’s Action Party', "People's Action Party",
       'Nominated Member of Parliament', "Workers' Party",
       'Progress Singapore Party', "Singapore People's Party"],
      dtype=object)

In [8]:
print(len(dataframe.loc[dataframe['Party'] == 'People’s Action Party']))
dataframe.loc[dataframe['Party'] == 'People’s Action Party', 'Party'] = "People's Action Party"
dataframe['Party'].unique()

65


array(['', "People's Action Party", 'Nominated Member of Parliament',
       "Workers' Party", 'Progress Singapore Party',
       "Singapore People's Party"], dtype=object)

1 row has 2 parties under the "Party" field. 

Treatment: represent as list

In [9]:
dataframe.loc[dataframe['Party'] == "People's Action Party, Barisan Sosialis"]

,Name,Party,Parliaments


In [10]:
dataframe = dataframe.assign(Party=dataframe.Party.str.split(', '))
dataframe.loc[dataframe['Name'] == 'Chan Sun Wing']

,Name,Party,Parliaments


In [11]:
dataframe

,Name,Party,Parliaments
0,Abdul Samad,[],"(14,)"
1,Alex Yam Ziming,[People's Action Party],"(12, 13, 14)"
2,Alvin Tan,[People's Action Party],"(14,)"
3,Alvin Yeo,[People's Action Party],"(11, 12)"
4,Amrin Amin,[People's Action Party],"(13,)"
...,...,...,...
181,Yip Pin Xiu,[Nominated Member of Parliament],"(13,)"
182,Zainal Sapari,[People's Action Party],"(12, 13)"
183,Zainudin Nordin,[People's Action Party],"(10, 11, 12)"
184,Zaqy Mohamad,[People's Action Party],"(11, 12, 13, 14)"


In [12]:
dataframe.to_csv('mps.csv', index=False)